In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import LocalOutlierFactor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import set_printoptions

# Reading the file

In [4]:
data = pd.read_csv('C:/Users/AJ/Documents/Workspace/Projects/Practice Proj/churn_treated.csv')
data.head()

,Unnamed: 0,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0,125.0,510.0,0.0,0.0,0.0,2013.4,99.0,28.66,1107.6,107.0,14.93,243.3,92.0,10.95,10.9,7.0,2.94,0.0,0.0
1,1,95.0,510.0,0.0,0.0,0.0,150.9,79.0,25.65,161.8,87.0,13.75,167.7,115.0,7.55,11.7,5.0,3.16,3.0,0.0
2,2,65.0,510.0,0.0,0.0,0.0,195.4,110.0,33.22,181.2,109.0,15.40,178.5,105.0,8.03,8.9,4.0,2.40,0.0,0.0
3,3,113.0,510.0,0.0,0.0,0.0,92.6,85.0,15.74,177.6,92.0,15.10,159.8,72.0,7.19,14.4,4.0,3.89,3.0,0.0
4,4,73.0,510.0,0.0,0.0,0.0,254.7,80.0,43.30,90.2,79.0,7.67,153.4,60.0,6.90,10.6,8.0,2.86,0.0,0.0


In [7]:
churn = data.loc[:, ['account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'churn']]
churn.head()

,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,125.0,510.0,0.0,0.0,0.0,2013.4,99.0,28.66,1107.6,107.0,14.93,243.3,92.0,10.95,10.9,7.0,2.94,0.0,0.0
1,95.0,510.0,0.0,0.0,0.0,150.9,79.0,25.65,161.8,87.0,13.75,167.7,115.0,7.55,11.7,5.0,3.16,3.0,0.0
2,65.0,510.0,0.0,0.0,0.0,195.4,110.0,33.22,181.2,109.0,15.40,178.5,105.0,8.03,8.9,4.0,2.40,0.0,0.0
3,113.0,510.0,0.0,0.0,0.0,92.6,85.0,15.74,177.6,92.0,15.10,159.8,72.0,7.19,14.4,4.0,3.89,3.0,0.0
4,73.0,510.0,0.0,0.0,0.0,254.7,80.0,43.30,90.2,79.0,7.67,153.4,60.0,6.90,10.6,8.0,2.86,0.0,0.0


In [8]:
churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3302 entries, 0 to 3301
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_length                 3302 non-null   float64
 1   area_code                      3302 non-null   float64
 2   international_plan             3302 non-null   float64
 3   voice_mail_plan                3302 non-null   float64
 4   number_vmail_messages          3302 non-null   float64
 5   total_day_minutes              3302 non-null   float64
 6   total_day_calls                3302 non-null   float64
 7   total_day_charge               3302 non-null   float64
 8   total_eve_minutes              3302 non-null   float64
 9   total_eve_calls                3302 non-null   float64
 10  total_eve_charge               3302 non-null   float64
 11  total_night_minutes            3302 non-null   float64
 12  total_night_calls              3302 non-null   f

In [10]:
churn.describe().T

,count,mean,std,min,25%,50%,75%,max
account_length,3302.0,97.330254,45.740927,-209.00,73.0000,100.0000,125.0000,232.00
area_code,3302.0,437.152938,42.348083,408.00,408.0000,415.0000,510.0000,510.00
international_plan,3302.0,0.097517,0.296705,0.00,0.0000,0.0000,0.0000,1.00
voice_mail_plan,3302.0,0.276499,0.447334,0.00,0.0000,0.0000,1.0000,1.00
number_vmail_messages,3302.0,7.262114,13.674976,-10.00,0.0000,0.0000,16.0000,51.00
total_day_minutes,3302.0,419.554588,618.342681,7.80,150.6000,191.2000,237.9750,2185.10
total_day_calls,3302.0,100.458207,19.506132,30.00,88.0000,101.0000,114.0000,165.00
total_day_charge,3302.0,30.676923,9.005898,1.33,24.6500,30.6825,36.7000,59.64
total_eve_minutes,3302.0,324.264345,313.177815,42.50,172.3250,210.1500,257.9250,1244.20
total_eve_calls,3302.0,100.081314,19.465360,36.00,87.0000,100.0000,113.0000,168.00


# Feature Selection

In [14]:
# Tuning for the number of features to select in modeling using RFE
aChurn = churn.values
a = aChurn[:, :-1]
b = aChurn[:, -1]

# get a list of models to evaluate
def getModels():
    models = dict()
    for i in range(2, 19):
        model = GradientBoostingClassifier() 
        rfe = RFE(model, n_features_to_select=i)
        models[str(i)] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

# evaluate a given model using cross-validation
def evaluateModel(model, a, b):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1) 
    scores = cross_val_score(model, a, b, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1) 
    return scores

# get the models to evaluate
models = getModels()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluateModel(model, a, b)
    results.append(scores)
    names.append(name)
    print('for %s features, Mean Absolute Error is: %.1f, Std is: %.3f' % (name, mean(scores) * 100, std(scores)))


for 2 features, Mean Absolute Error is: -12.4, Std is: 0.012
for 3 features, Mean Absolute Error is: -9.7, Std is: 0.012
for 4 features, Mean Absolute Error is: -10.1, Std is: 0.014
for 5 features, Mean Absolute Error is: -8.8, Std is: 0.015
for 6 features, Mean Absolute Error is: -6.8, Std is: 0.013
for 7 features, Mean Absolute Error is: -5.2, Std is: 0.011
for 8 features, Mean Absolute Error is: -4.9, Std is: 0.009
for 9 features, Mean Absolute Error is: -4.9, Std is: 0.010
for 10 features, Mean Absolute Error is: -4.9, Std is: 0.011
for 11 features, Mean Absolute Error is: -4.9, Std is: 0.010
for 12 features, Mean Absolute Error is: -4.9, Std is: 0.010
for 13 features, Mean Absolute Error is: -5.1, Std is: 0.010
for 14 features, Mean Absolute Error is: -5.0, Std is: 0.010
for 15 features, Mean Absolute Error is: -5.1, Std is: 0.011
for 16 features, Mean Absolute Error is: -5.1, Std is: 0.011
for 17 features, Mean Absolute Error is: -5.0, Std is: 0.010
for 18 features, Mean Absolute

Selecting  8 features gives us the closest value to 0 at -4.9 in Mean Absolute Error and a std of 0.009. This makes 8 features the best for feature selection.

In [12]:
# Identfying the algorithm that would select feature better
# get the dataset
fChurn = churn.values
a = fChurn[:, :-1]
b = fChurn[:, -1]

# get a list of models to evaluate
def getModels():
    models = dict()
    
    # logistic regression
    model = LogisticRegression()
    rfe = RFE(model, n_features_to_select=8) # for regression model use LinearRegression
    models['logistic regression'] = Pipeline(steps=[('s',rfe),('m',model)])
    
    # perceptron
    model = Perceptron()
    rfe = RFE(model, n_features_to_select=8)
    models['perceptron'] = Pipeline(steps=[('s',rfe),('m',model)])
    
    # decision tree
    model = DecisionTreeClassifier()
    rfe = RFE(model, n_features_to_select=8) # for regression model use DecisionTreeRegressor
    models['decision tree'] = Pipeline(steps=[('s',rfe),('m',model)])
    
    # random forest
    model = RandomForestClassifier()
    rfe = RFE(model, n_features_to_select=8) # for regression model use RandomForestRegressor
    models['random forest'] = Pipeline(steps=[('s',rfe),('m',model)])
    
    # gradient boosting 
    model = GradientBoostingClassifier()
    rfe = RFE(model, n_features_to_select=8) # for regression model use GradientBoostingRegressor
    models['gradient boosting'] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

# evaluate a given model using cross-validation
def evaluateModel(model, a, b):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, a, b, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = getModels()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluateModel(model, a, b)
    results.append(scores)
    names.append(name)
    print('the %s algorithm has mae of %.3f and a Std of %.3f' % (name, mean(scores)*100, std(scores)))


the logistic regression algorithm has mae of -13.921 and a Std of 0.014
the perceptron algorithm has mae of -43.677 and a Std of 0.298
the decision tree algorithm has mae of -8.076 and a Std of 0.012
the random forest algorithm has mae of -7.197 and a Std of 0.010
the gradient boosting algorithm has mae of -5.038 and a Std of 0.009


The Gradient boosting Algorithm's Mean Absolute Error has the closest value to 0 at -5.028. This makes it the best algorthim to use for feature selection

Feature selection using Recursive Feature Elimination (RFE) with 8 features

In [17]:
fsChurn8= churn.values
c = fsChurn8[:, 0:-1]
d = fsChurn8[:, -1]

# feature extraction
model = GradientBoostingClassifier()

rfe = RFE(model, n_features_to_select=8)
pipeline = Pipeline(steps=[('s',rfe), ('m',model)])
fit = rfe.fit(c, d)

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
nScores = cross_val_score(pipeline, c, d, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# report performance
print('Mean Absolute Error: %.3f '  %  (mean(nScores)))
print()
print('Standard Deviation Accuracy: %.3f' % std(nScores))
print()
print('Selected Features: {}'.format(fit.n_features_))

Mean Absolute Error: -0.051 

Standard Deviation Accuracy: 0.011

Selected Features: 8


Features that were selected

In [19]:
features = rfe.transform(c)

pChurn = pd.DataFrame(features)
pChurn.columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

# to make it look organised when returning 
chuCol = dict()
cols = list(churn.columns)[0:-1] # the list of columns in your data
for col in cols:
    chuCol[col]= churn[col].values.tolist()
    
collen = len(pChurn.columns.tolist())
counter = 0
while counter < collen:
    colLabel = ''
    if counter == 0:
        colLabel = 'a'
    elif counter == 1:
        colLabel = 'b'
    elif counter == 2:
        colLabel = 'c'
    elif counter == 3:
        colLabel = 'd'  
    elif counter == 4:
        colLabel = 'e'
    elif counter == 5:
        colLabel = 'f'
    elif counter == 6:
        colLabel = 'g'
    elif counter == 7:
        colLabel = 'h'
    else:
        pass
    colData = pChurn.loc[:,colLabel]
    colDataVals = list(colData.values)
    for item in chuCol.items():
        if colDataVals == item[1]:
            print('Label is: ', item[0])
    counter+=1
print()
# end of organisation

print('The selected features are: ')
pChurn.head(10)

Label is:  international_plan
Label is:  voice_mail_plan
Label is:  total_day_charge
Label is:  total_eve_charge
Label is:  total_intl_minutes
Label is:  total_intl_calls
Label is:  total_intl_charge
Label is:  number_customer_service_calls

The selected features are: 


,a,b,c,d,e,f,g,h
0,0.0,0.0,28.66,14.93,10.9,7.0,2.94,0.0
1,0.0,0.0,25.65,13.75,11.7,5.0,3.16,3.0
2,0.0,0.0,33.22,15.40,8.9,4.0,2.40,0.0
3,0.0,0.0,15.74,15.10,14.4,4.0,3.89,3.0
4,0.0,0.0,43.30,7.67,10.6,8.0,2.86,0.0
5,0.0,0.0,27.27,18.54,12.5,4.0,3.38,1.0
6,0.0,0.0,25.21,15.44,8.3,6.0,2.24,3.0
7,0.0,0.0,41.16,16.24,14.7,2.0,3.97,3.0
8,0.0,0.0,32.66,21.23,7.3,3.0,1.97,3.0
9,0.0,0.0,37.62,13.41,11.3,6.0,3.05,1.0


In [20]:
# attaching the feature names to the dataset
pChurn.columns = ['international_plan', 'voice_mail_plan', 'total_day_charge', 'total_eve_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls']
pChurn['churn'] = churn['churn']
pChurn.head()

,international_plan,voice_mail_plan,total_day_charge,total_eve_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
0,0.0,0.0,28.66,14.93,10.9,7.0,2.94,0.0,0.0
1,0.0,0.0,25.65,13.75,11.7,5.0,3.16,3.0,0.0
2,0.0,0.0,33.22,15.40,8.9,4.0,2.40,0.0,0.0
3,0.0,0.0,15.74,15.10,14.4,4.0,3.89,3.0,0.0
4,0.0,0.0,43.30,7.67,10.6,8.0,2.86,0.0,0.0


In [21]:
pChurn.describe()

,international_plan,voice_mail_plan,total_day_charge,total_eve_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
count,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000
mean,0.097517,0.276499,30.676923,17.066184,10.226423,4.461791,2.761644,1.563295,0.144458
std,0.296705,0.447334,9.005898,4.182815,2.769781,2.406992,0.747805,1.295685,0.351607
min,0.000000,0.000000,1.330000,3.610000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,24.650000,14.202500,8.500000,3.000000,2.300000,1.000000,0.000000
50%,0.000000,0.000000,30.682500,17.090000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,0.000000,1.000000,36.700000,19.857500,12.000000,6.000000,3.240000,2.000000,0.000000
max,1.000000,1.000000,59.640000,30.910000,20.000000,20.000000,5.400000,9.000000,1.000000


# Data Transformation

Rescaling the selected features

In [22]:
rPChurn = pChurn.values
g= rPChurn[:, 0:-1]
h= rPChurn[:, -1]

scaler = MinMaxScaler()
rpc = scaler.fit_transform(g)
set_printoptions(precision=3)

# put transorm data into dataframe and display
rChurn = pd.DataFrame(rpc)
rChurn.columns = ['international_plan', 'voice_mail_plan', 'total_day_charge', 'total_eve_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls']

rChurn['churn'] = pChurn['churn']
rChurn.describe()


,international_plan,voice_mail_plan,total_day_charge,total_eve_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
count,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.00000,3302.000000,3302.000000,3302.000000
mean,0.097517,0.276499,0.503291,0.492901,0.511321,0.22309,0.511416,0.173699,0.144458
std,0.296705,0.447334,0.154449,0.153217,0.138489,0.12035,0.138482,0.143965,0.351607
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.399931,0.388004,0.425000,0.15000,0.425926,0.111111,0.000000
50%,0.000000,0.000000,0.503387,0.493773,0.515000,0.20000,0.514815,0.111111,0.000000
75%,0.000000,1.000000,0.606585,0.595147,0.600000,0.30000,0.600000,0.222222,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


Normalizing the rescaled features

In [23]:
# Normalizing the data

norm = rChurn.values
i = norm[:, 0:-1]
j = norm[:, -1]

nScaler = Normalizer()
nh = nScaler.fit_transform(i)
set_printoptions(precision=3)

# put transorm data into dataframe and display
nChurn = pd.DataFrame(nh)
nChurn.columns = ['international_plan', 'voice_mail_plan', 'total_day_charge', 'total_eve_charge', 'total_intl_minutes', 'total_intl_calls', 'total_intl_charge', 'number_customer_service_calls']

nChurn['churn'] = rChurn['churn']
nChurn.describe()

,international_plan,voice_mail_plan,total_day_charge,total_eve_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churn
count,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000,3302.000000
mean,0.062066,0.184202,0.418397,0.409573,0.420536,0.185956,0.420617,0.144734,0.144458
std,0.190247,0.300201,0.135311,0.133411,0.107253,0.101556,0.107256,0.120149,0.351607
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.322105,0.317023,0.349706,0.112483,0.350018,0.072147,0.000000
50%,0.000000,0.000000,0.415499,0.404420,0.426047,0.167142,0.426345,0.116391,0.000000
75%,0.000000,0.572883,0.505363,0.494843,0.502662,0.240617,0.502678,0.214092,0.000000
max,0.855284,0.903374,0.939826,0.876649,0.667839,0.745364,0.668453,0.674733,1.000000


Baseline Performance of the original data

In [25]:
chur = churn.values
k= chur[:, 0:-1]
l= chur[:, -1]

# define and configure the model
model = KNeighborsClassifier()

# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
nScores = cross_val_score(model, k, l, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# report model performance
print('Baseline Mean Accuracy: %.3f' % (mean(nScores)*100))
print('Standard Deviation Accuracy: %.3f' % std(nScores))

Baseline Mean Accuracy: -12.962
Standard Deviation Accuracy: 0.009


Performance of the normalised features selected for modeling

In [26]:
# performance of rescaled data
nChur = nChurn.values
m= nChur[:, 0:-1]
n= nChur[:, -1]

# define and configure the model
model = KNeighborsClassifier()

# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
nScores = cross_val_score(model, m, n, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# report model performance
print('Normalized data Mean Accuracy: %.3f' % (mean(nScores)*100))
print('Standard Deviation Accuracy: %.3f' % std(nScores))

Normalized data Mean Accuracy: -12.902
Standard Deviation Accuracy: 0.013


Testing for the accuracy of the rescaled selected features with model

In [33]:
tChurn = nChurn.values
x = tChurn[:, 0:-1]
y = tChurn[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

model = RandomForestClassifier()
model.fit(x_train, y_train) 

# evaluate the model
yhat = model.predict(x_test)

# evaluate predictions
accuracy = accuracy_score(y_test, yhat)
print('The accuracy of the selected features for modeling is: %.2f' % (accuracy*100))

The accuracy of the selected features for modeling is: 89.91
